# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Chroma

In [ ]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

from langchain.vectorstores.chroma import Chroma
from dotenv import load_dotenv, find_dotenv
import os
# # 从环境变量中加载你的 API_KEY
# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../data_base/vector_db/chroma-vmax'

# # 创建嵌入模型
# from langchain_community.embeddings import ZhipuAIEmbeddings

# zhipu_embed = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key=zhipuai_api_key
# )

from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434',model="bge-m3:latest")

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        collection_name="vmax-s",
        embedding_function=my_emb
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [ ]:
question = "VMAX上网日志业务是什么？"
docs = vectordb.similarity_search(question,k=5)
print(f"检索到的内容数：{len(docs)}")

打印一下检索到的内容

In [ ]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

### Milvus

In [1]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

C:\Users\will\AppData\Local\Temp\ipykernel_23612\3633838239.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
C:\Users\will\AppData\Local\Temp\ipykernel_23612\3633838239.py:6: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectordb = Milvus(


In [ ]:
results = vectordb.similarity_search(query="VMAX", k=2)
# results

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [2]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://localhost:11434', model='deepseek-r1:14b', temperature=0.1)

# my_llm.invoke("你好")

C:\Users\will\AppData\Local\Temp\ipykernel_23612\73905523.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  my_llm = Ollama(base_url='http://localhost:11434', model='qwen2.5:0.5b', temperature=0.1)


## 3. 构建检索问答链

prompts

In [3]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [4]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


## 4.检索问答链效果测试

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [5]:
questions = [
    "什么是VMAX？",
    "有哪些功能", 
    # "整理成表格"  
]

for question in questions:
    result = qa_chain({"query": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['result']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

C:\Users\will\AppData\Local\Temp\ipykernel_23612\2682531927.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})  # Pass string directly, not dict


回答：根据您提供的信息，VMAX（Voice and Media Services）是联通集团研发的语音和媒体服务系统。以下是关于VMAX的一些关键点：

1. **功能**:
   - 支持对移动通信网络、KPI（关键性能指标）、业务等多个维度进行评估。
   - 为运营商提供咨询服务，包括网络评估咨询、网络发展咨询等。

2. **特点**:
   - 采用分布式存储集群和本地分布式文件系统相结合的架构。
   - 配备防火墙日志处理单元、数据合成单元和客户端访问接口。
   - 支持MPP数据库Gbase作为核心数据库，支持多用户并发查询。

3. **应用场景**:
   - 对移动通信网络进行评估和分析，确定其现状和存在的主要矛盾。
   - 根据多模网络的负荷分布、终端能力、价值用户分布等信息，挖掘价值区域，为运营商的网络建设提供指导。

4. **优势**:
   - 采用分布式存储集群和本地分布式文件系统相结合的架构，提高了系统的可扩展性和可靠性。
   - 配备防火墙日志处理单元和数据合成单元，确保了系统的安全性。
   - 支持MPP数据库Gbase作为核心数据库，支持多用户并发查询。

5. **技术背景**:
   - 该系统是联通集团在2018年推出的一项语音和媒体服务解决方案，旨在提升运营商的网络服务质量。

VMAX系统通过这些功能和技术，为运营商提供了一个全面、高效的评估和分析工具，帮助运营商更好地理解和优化其移动通信网络。


回答：4个主要功能：

1. 用户分析功能：用于用户级的在某一时间段的呼叫、注册记录，媒体数据、信令详情，并结合最近7天的呼叫、注册统计情况，分析定位语音/视频通话质量发生异常的环节。

2. 网络分析功能：支持简化系统的安装部署、开通参数配置以及系统运行状态健康巡查。系统支持一键式安装、开通和数据质量管理。

3. 专题分析功能：实时跟踪设定用户的消息信令流程，及时分析用户业务质量，并帮助运营商定位业务过程中的问题和处理用户可能的投诉。




In [ ]:
question_1 = "什么是vmax的上网日志系统？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

In [ ]:
question_2 = "严威是谁？"
result = qa_chain({"query": question_21})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

### 4.2 无知识库大模型自己回答的效果

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
my_llm.predict(prompt_template)

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
my_llm.predict(prompt_template)

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [6]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

C:\Users\will\AppData\Local\Temp\ipykernel_23612\2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
# 修改后的Prompt模板（添加chat_history变量）
template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
# 创建对话链
qa_chain = ConversationalRetrievalChain.from_llm(
        llm=my_llm,
        retriever=vectordb.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": QA_PROMPT},
        chain_type="stuff"
    )
    
    # result = qa_chain({"question": question})
    # return result["answer"]
    

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
questions = [
    "什么是VMAX？",
    "有哪些功能", 
    # "整理成表格"  
]

for question in questions:
    result = qa_chain({"question": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

回答：VMAX是ValueMAX端到端多维价值分析系统的简称，面向用户的网络运维和运营分析产品。它立足于从客户角度去感知和分析网络和业务信息，并通过实时的分析、事后分析等方式为运营商提供全面支撑。


